### <p style="text-align: right;"> &#9989; Andrew </p>

# PHY480 Day 10

## In-class assignment: Linear algebra, Gaussian elimination and LU decomposition

In this in-class assignment we discuss methods for solving systems of linear equations: Guassian elimination and LU decomposition. These methods are similar, however, the Gaussian elimination method solves a system $Ax=b$, while LU decomposition uses a similar process to decompose $A$ as a product $A=LU$, where $L$ is a lower-triangular and $U$ is an upper-triangular matrices with the intent of speeding up solving $Ax=b$ systems later. The latter method is beneficial if one anticipates solving a number of similar systems that have the same matrix $A$ but differ in the right-hand-side vector $b$.

Calculating a matrix inverse is a similar process, since it is equivalent to solving a collection of systems $Ax=b$ where the vectors $b$ form an identity matrix.


In [11]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


**Task 1.** Implement the forward and backward substitution code from the book of Gezerlis, chapter 4 (code 4.1 in 2023 edition).

If my answers look familiar, they should be!

In [12]:
# solve lower-triangular system Lx=b by forward substitution (no pivoting)
# Input:
# L -- square lower-triangular matrix
# bs -- right-hand-side vector
# Output:
# solution vector
def forsub(L,bs):
    n = bs.size
    xs = np.zeros(n)
    for i in range(n):
        val = 0.
        for j in range(i):
            val += L[i,j]*xs[j]
        xs[i] = (bs[i] - val)/L[i,i]
    return xs


# solve upper-triangular system Ux=b with backward substitution (no pivoting)
# Input:
# U -- square upper-triangular matrix
# bs -- right-hand-side vector
# Output:
# solution vector
def backsub(U,bs):
    n = bs.size
    xs = np.zeros(n)
    for i in reversed(range(n)):
        val = 0.
        for j in range(i+1,n):
            val += U[i,j]*xs[j]
        xs[i] = (bs[i] - val)/U[i,i]
    return xs


**Task 2.** Implement the Gaussian elimination code from the book of Gezerlis, chapter 4 (codes 4.2 and 4.4 in the 2023 edition).


In [13]:
# solve Ax=b by Gaussian elimination (no pivoting)
# Input:
# inA -- square matrix A
# inbs -- right-hand-side vector
# Output:
# solution vector
def gauelim( inA, inbs ):
    A = np.copy(inA)
    bs = np.copy(inbs)
    n = bs.size

    for j in range(n-1):
        for i in range(j+1,n):
            coeff = A[i,j]/A[j,j]
            A[i,j:] -= coeff*A[j,j:]
            bs[i] -= coeff*bs[j]

    xs = backsub(A,bs)
    return xs


# solve Ax=b by Gaussian elimination with partial pivoting
# Input:
# inA -- square matrix A
# inbs -- right-hand-side vector
# Output:
# solution vector
def gauelim_pivot(inA,inbs):
    A = np.copy(inA)
    bs = np.copy(inbs)
    n = bs.size

    for j in range(n-1):
        k = np.argmax(np.abs(A[j:,j])) + j
        if k != j:
            A[j,:], A[k,:] = A[k,:], A[j,:].copy()
            bs[j], bs[k] = bs[k], bs[j]

        for i in range(j+1,n):
            coeff = A[i,j]/A[j,j]
            A[i,j:] -= coeff*A[j,j:]
            bs[i] -= coeff*bs[j]

    xs = backsub(A,bs)
    return xs


**Task 3.** Implement the LU decomposition and linear system solver code from the book of Gezerlis, chapter 4 (code 4.3 in the 2023 edition).

In [14]:
# LU decomposition of A (no pivoting)
# Input:
# A -- square matrix
# Output:
# L, U -- lower- and upper-triangular parts
def ludec( A ):
    n = A.shape[0]
    U = np.copy(A)
    L = np.identity(n)

    for j in range(n-1):
        for i in range(j+1,n):
            coeff = U[i,j]/U[j,j]
            U[i,j:] -= coeff*U[j,j:]
            L[i,j] = coeff

    return L, U


# solve linear system by using LU decomposition
# Input:
# A -- square matrix
# bs -- right-hand-side vector
# Output:
# solution vector
def lusolve( A, bs ):
    L, U = ludec(A)
    ys = forsub(L,bs)
    xs = backsub(U,ys)
    return xs


**Task 4.** Generate random matrices $A$ of different sizes and the right-hand-side vectors $b$ (of matching size) and solve the linear system $Ax=b$ with your implementation of the Gaussian elimination algorithm (with and without pivoting). Compare your solution to the one calculated with `numpy.linalg.solve`.

**Note:** In general, chances are a random matrix is singular (i.e. $Ax=b$ does not have a solution and $A$ does not have an inverse). In that case, simply change the random seed.


In [30]:
# it is a good idea to fix the random seed for reproducibility
np.random.seed( 1 )

n = 5 # experiment with different sizes from e.g. 2 to 12

A = np.random.rand( n, n )
b = np.random.rand( n )

print( A )


[[4.17022005e-01 7.20324493e-01 1.14374817e-04 3.02332573e-01
  1.46755891e-01]
 [9.23385948e-02 1.86260211e-01 3.45560727e-01 3.96767474e-01
  5.38816734e-01]
 [4.19194514e-01 6.85219500e-01 2.04452250e-01 8.78117436e-01
  2.73875932e-02]
 [6.70467510e-01 4.17304802e-01 5.58689828e-01 1.40386939e-01
  1.98101489e-01]
 [8.00744569e-01 9.68261576e-01 3.13424178e-01 6.92322616e-01
  8.76389152e-01]]


In [31]:
# YOUR CODE HERE

lusolve(A, b)

array([-1.14572898,  2.35642506,  0.08995417, -1.28158098,  0.42562896])

**Task 5.** Repeat the previous exercise with your version of the LU decomposition-based solver. Compare the result with `numpy.linalg.solve`.

In [18]:
# YOUR CODE HERE

np.linalg.solve(A, b)

array([-1.14572898,  2.35642506,  0.08995417, -1.28158098,  0.42562896])

**Task 6.** Based on your LU decomposition, implement calculation of matrix inverse. (Make sure that LU decomposition is called only once during that calculation!) After calculating the inverse, check that $A^{-1}A=I$ (approximately) holds. Also, compare your inverse with the result of `numpy.linalg.inv`.

In [82]:
# calculate matrix inverse using LU decomposition (no pivoting)
# Input:
# A -- square matrix
# Output:
# inverse matrix
def mat_inverse(A):
    n = A.shape[0]
    L, U = ludec(A)
    I = np.eye(n)
    inv_A = np.zeros((n, n))
    for i in range(n):
        Y = forsub(L, I[:, i])
        X = backsub(U, Y)
        inv_A[:, i] = X

    return inv_A

In [87]:
X = mat_inverse(A)
A_inv = np.linalg.inv(A)

assert np.max(A_inv - X) < 1e-10

&#169; Copyright 2025,  Michigan State University Board of Trustees